In [2]:
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd

In [3]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [4]:
Default_Column_ZSD76=['Solic.', 'Solicitante', 'Fecha salida mercancía', 'Material',
       'Número de material', 'Cantidad entrega', 'UM', 'MARCA', 'CATEGORIA',
       'Entrega', 'ClEnt', 'Clase de entrega', 'ClVt',
       'Clase de documento de ventas', 'OrgVt', 'CDis', 'Doc.venta',
       'Prc.neto', 'Mon.', 'Destinat.', 'Destinatario de mercancías',
       'Nº pedido cliente', 'Fecha creación Pedido']

Default_Column_ZS127=['Cliente', 'Nombre', 'Pedido Externo - OC', 'Pedido', 'Tipo',
       'Denominación', 'Fecha creación Ped.', 'Status Picking (SM)',
       'Status Factura', 'Status Transporte', 'Transporte']

In [5]:
transsacion_ZSD76='zsd76'
variant_ZSD76="JESPINOSAP"
provision_ZSd76="PROCESADOS"

In [5]:
def Search_table_Variant(session,Variant):
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    variants =session.findById("wnd[1]/usr/cntlALV_CONTAINER_1/shellcont/shell")
    row=variants.RowCount  #Count total rows
    lis=[variants.GetCellValue(i,"VARIANT") for i in range(0,row)]  #send variant apply GetCellValue
    indice=[indice for indice, dato in enumerate(lis) if dato == Variant]
    variants.selectedRows = indice[0]
    session.findById("wnd[1]/tbar[0]/btn[2]").press()
    return(Variant)

In [6]:
def Search_ZSD76(Transsaccion,Variant,session):
    session.StartTransaction(Transsaccion)
    Search_table_Variant(session,Variant)
    session.findById("wnd[0]/tbar[1]/btn[8]").press()

In [7]:
def Search_ZSD127(Transsaccion,Variant,session):
    session.StartTransaction(Transsaccion)
    Search_table_Variant(session,Variant)
    session.findById("wnd[0]/tbar[1]/btn[8]").press()

In [8]:
def Search_ZSD76(Transsaccion,session,provision,variant):
    session.StartTransaction(Transsaccion)
    session.findById("wnd[0]/tbar[1]/btn[17]").press()
    session.findById("wnd[1]/usr/txtV-LOW").text = provision
    session.findById("wnd[1]/usr/txtENAME-LOW").text = variant
    session.findById("wnd[1]/usr/txtV-LOW").caretPosition = 8
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    session.findById("wnd[0]/tbar[1]/btn[8]").press()
    return(session)

In [7]:
SAP_GUI.Search_ZSD76(transsacion_ZSD76,session,provision_ZSd76,variant_ZSD76)

AttributeError: module 'Librerias_SAP.SAP_GUI' has no attribute 'Search_ZSD76'

In [54]:
#Dowloader data

Name_ZSD76="Detalle_Entregas_ZSD76"
Ruta_ZSD76="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD76,session,Ruta_ZSD76)

In [55]:
Pedidos_Procesados=pd.read_csv(Ruta_ZSD76+Name_ZSD76+".txt",delimiter="\t",skiprows=5)
Pedidos_Procesados=Funtions.Clean_Columns(Pedidos_Procesados)
Pedidos_Procesados=Funtions.default_column(Default_Column_ZSD76,Pedidos_Procesados)

In [91]:
Transsaccion_ZSD127="zsd127"
Variant_ZSD127="ESTANDAR"
Documento_Ventas=Pedidos_Procesados['Doc.venta'].drop_duplicates()
Series=Documento_Ventas
provision=None

In [124]:
def Search_Pedidos_ZSD127(Transsaccion,Series,session,Variant,provision=None):      #(column Dataframe)
    """
    Transsaccion: Transsacion a buscar
    Series: Columna del dataframe que quiero copiar
    session: session del usuario
    Variant: Variante de visualizacion
    provision: disposicion o Layout
    """
    session.StartTransaction(Transsaccion)
    Search_table_Variant(session,Variant)
    Series=Series.to_clipboard(index=False, header=False)
    session.findById("wnd[0]/usr/btn%_SP$00004_%_APP_%-VALU_PUSH").press()
    session.findById("wnd[1]/tbar[0]/btn[24]").press()
    session.findById("wnd[1]/tbar[0]/btn[8]").press()
    if provision!=None:
        session.findById("wnd[0]/usr/ctxt%LAYOUT").text = provision
    session.findById("wnd[0]/tbar[1]/btn[8]").press()

In [125]:
SAP_GUI.Search_Pedidos_ZSD127(Transsaccion_ZSD127,Documento_Ventas,session,Variant_ZSD127,provision=None)

In [111]:
#Dowloader data

Name_ZSD127="Detalle_Entregas_ZSD76"
Ruta_ZSD127="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD76,session,Ruta_ZSD76)

In [116]:
Transporte_Alertas=pd.read_csv(Ruta_ZSD127+Name_ZSD127+".txt",delimiter="\t",skiprows=5)
Transporte_Alertas=Funtions.Clean_Columns(Transporte_Alertas)
#Transporte_Alertas=Funtions.default_column(Default_Column_ZSD76,Pedidos_Procesados)